# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        # self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.classical_gain = self.__classical_gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.classical_gain = self.__classical_entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.classical_gain = self.__classical_misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __classical_gini(self, y):
        return 1 - ((np.bincount(y)/y.size)**2).sum()

    def __classical_entropy(self, y):
        p = np.bincount(y) / y.size + 1e-9
        return -(p * np.log2(p)).sum()

    def __classical_misclass(self, y):
        return 1 - (np.bincount(y) / y.size).max()

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return -((l_c ** 2).sum(axis=1) / np.squeeze(l_s) +
                 (r_c ** 2).sum(axis=1) / np.squeeze(r_s))

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(l_c * (l_c / l_s + 1e-9) *
                 np.log2(l_c / l_s + 1e-9) + r_c * (r_c / r_s + 1e-9)
                 * np.log2(r_c / r_s + 1e-9)).sum(axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return -(r_c).max(axis=1) - (l_c).max(axis=1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, X, y, feature_id, threshold):
        # left_mask = x[:, feature_id] > threshold
        # right_mask = ~left_mask
        # return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
        left_mask = X[:, feature_id] > threshold
        equal_mask = X[:, feature_id] == threshold
        right_mask = X[:, feature_id] < threshold
        X_l = X[left_mask]
        X_r = X[right_mask]
        y_l = y[left_mask]
        y_r = y[right_mask]
        if y_r.size == 0 and y_l.size == 0:
            X_l = X[:np.int(X.shape[0] / 2)]
            y_l = y[:np.int(X.shape[0] / 2)]
            X_r = X[-np.int(X.shape[0] / 2):]
            y_r = y[-np.int(X.shape[0] / 2):]
            return X_l, X_r, y_l, y_r
        if y_r.size == 0:
            X_r = X[equal_mask]
            y_r = y[equal_mask]
            return X_l, X_r, y_l, y_r
        if y_l.size == 0:
            X_l = X[equal_mask]
            y_l = y[equal_mask]
            return X_l, X_r, y_l, y_r
        X_r = X[right_mask | equal_mask]
        y_r = y[right_mask | equal_mask]
        return X_l, X_r, y_l, y_r

    def __find_threshold(self, x, y):
        # функция find_threshold нужна для поиска оптимальногo
        # признака, по которому произойдет разбиение:
        # принимает столбец определенного признака и столбец целевой переменной
        # возвращает impurity(нужно минимизировать) и threshold
        # (по нему нужно разбить объекты на 2 класса)

        # Что делает этот блок кода?
        # sort_samples y по ключу х
        # число различных классов записывается в переменную class_number
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # class_number = np.unique(y).size

        # допустим, что np.unique выдаст [1]
        # тогда num_classes -> 1
        # при использовании one hot encoding далее при попытке
        # обращенияпо индексу sorted_y[r_border_ids - 1] -> 1
        # алгоритм выйдет за границу массива. чтобы этого не произошло,
        # следует использовать другое определение:
        class_number = y.max() + 1

        # Что делает этот блок кода?
        # выделяем центральную часть y_sorted (отсекаем по
        # min_samples_split слева и справа,
        # так как их нельзя делить)
        # и записываем результат в splitted_sorted_y
        # конструкция внутри np.where сравнивает сдвинутые
        # на 1 значения массива splitted_sorted_y
        # [x * * * * *]
        #   [* * * * * x]
        # таким образом, мы получим индексы массива splitted_sorted_y,
        # после которых следует элемент другого класса
        # для получения индекса элемента, на котором меняется класс,
        # прибавляем 1, чтобы вернуться к адресации
        # в индексах массива sorted_y, прибавляем min_samples_split
        cut_size = self.min_samples_split // 2 - 1
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1]
                                != splitted_sorted_y[1:])[0] + (cut_size + 1)

        # r_borders_ids содержит индексы, начиная с
        # которых в sorted_y идут объекты другого класса

        # если получили, что в данной выборке
        # все элементы одного класса, разбивать не надо
        if len(r_border_ids) == 0:
            return float('+inf'), None

        # Что делает этот блок кода?
        # eq_el_count - количество элементов одного класса,
        # идущих подряд (вектор из размеров кластеров)
        # len(eq_el_count) - количество кластеров в sorted_y
        eq_el_count = r_border_ids - np.append([cut_size], r_border_ids[:-1])
        # создаем матрицу one_hot_code. число строк: количество кластеров,
        # число столбцов: количество классов
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        # ставим единицу в том столбце класса,
        # к которому относится данных кластер
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        # домножаем получившуюся матрицу посимвольно на столбец,
        # содержащий размер кластеров
        # таким образом, class_increments - это матрица.
        # каждая ее строка - это отдельный кластер,
        # индекс ненулевого столбца показывает, к какому
        # классу относится данный кластер, а значение - размер кластера
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # мы отрезали cut_size элементов, так как не может делить
        # в той области, но они есть, их надо учитывать в матрице
        class_increments[0] = class_increments[0] + np.bincount(
            sorted_y[:cut_size], minlength=class_number)

        # Что делает этот блок кода?
        # считаем кумулятивную сумму по столбцам матрицы
        # i-я строка показывает, сколько элементов каждого класса
        # встретилость от i = 0 до текущего
        # np.bincount(sorted_y) содержит общую статистику
        # входного столбца целевой переменной
        # l_class_count[i] показывает, сколько элементов каждого
        # класса будет в левой ноде, если мы сделаем разбиение на индексе i
        # тогда в правую ноду попадет все - l_class_count[i]
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y) - l_class_count
        # l_sizes[i] - число элементов в левой ноде,
        # если разбивать по элементу с индексом i
        # соответственно r_sizes = все - l_sizes
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # вычисляем impurity для каждого из таких разбиений
        # локальная оптимизация: impurity должна быть минимальной
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # выбираем id элемента, по которому будем бить (он попадет в правую
        # ноду). так как l_sizes содержит в себе кумулятивно накопленные
        # размеры кластеров, то l_sizrd[idx] как раз будет показывать
        # сколько элементов в левой ноде, а сколько в правой
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __make_leaf(self, node_id, y):
        prediction = np.bincount(y).argmax()
        probs = np.bincount(y) / y.size
        self.tree[node_id] = (self.LEAF_TYPE, prediction, probs)

    def __fit_node(self, X, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node

        if (depth ==
            self.max_depth) or (np.bincount(y).max() / y.size
                                > self.sufficient_share):
            self.__make_leaf(node_id, y)
            return
        f_id = self.get_feature_ids(X.shape[1])
        thresholds = np.array([self.__find_threshold(X[:, id], y)
                               for id in f_id])
        best_id = thresholds[:, 0].argmin()
        _, threshold = thresholds[best_id]
        if threshold is None:
            self.__make_leaf(node_id, y)
            return
        X_l, X_r, y_l, y_r = self.__div_samples(X, y, f_id[best_id], threshold)
        self.tree[node_id] = (self.NON_LEAF_TYPE, f_id[best_id], threshold)
        self.feature_importances_[best_id] += self.classical_gain(
            y) - (y_l.size * self.classical_gain(y_l) +
                  y_r.size * self.classical_gain(y_r)) / y.size
        self.__fit_node(X_l, y_l, 2 * node_id + 1, depth + 1, pred_f)
        self.__fit_node(X_r, y_r, 2 * node_id + 2, depth + 1, pred_f)

    def fit(self, X, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(X.shape[1])
        self.__fit_node(X, y, 0, 0)
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

Wall time: 3.13 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

Wall time: 24 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.77661227661227661

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.78354978354978355

## Подготовка данных Speed Dating Data 

In [9]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)
df.drop_duplicates('iid').age.isnull().sum()
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
pd.get_dummies(df, columns=['field_cd'], 
                prefix='field_cd', prefix_sep='=')

df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
df = df.drop(['sports','tvsports','exercise','dining','museums','art','hiking','gaming',
       'clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga'], axis=1)
df = df.drop(['expnum'], axis=1)
feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
(df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100
feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), 
                       on='pid', 
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [11]:
# тут должен быть код типа 
%time clf.fit(X_train, y_train)

Wall time: 141 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [12]:
# тут должен быть код типа
%time my_clf.fit(X_train, y_train)

Wall time: 9.73 s


## Проверка качества работы на Speed Dating Data

In [13]:
# тут должен быть код типа 
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.59376554468579013

In [14]:
# тут должен быть код типа 
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5937908022631655

## Задание 3

In [15]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('clf', DecisionTreeClassifier()),
])

pipeline.fit(X_train, y_train)
f1_score(y_pred=pipeline.predict(X_test), y_true=y_test, average='macro')

0.57525335312735182

## Задание 4

In [16]:
from tabulate import tabulate

def make_stat(names, clf, n, title):
    importances = clf.feature_importances_
    top_n = importances.argsort()[-n:][::-1]
    table = np.array([names[top_n], importances[top_n]])
    print('\n', title, '\n')
    print(tabulate(table.T, headers=['feature', 'importance'], tablefmt='orgtbl'))
    
names = df_pair.columns[1:]
make_stat(names, my_clf, 10, 'my_clf')
make_stat(names, clf, 10, 'sklearn')


 my_clf 

| feature    |   importance |
|------------+--------------|
| int_corr   |  0.0171645   |
| samerace   |  0.00492841  |
| age        |  0.00180618  |
| field_cd_f |  0.0012095   |
| field_cd   |  0.00113858  |
| age_f      |  0.000912726 |
| imprace_f  |  0.000636512 |
| mn_sat     |  0.000562853 |
| income_f   |  0.000466786 |
| shar1_1_f  |  0.000435938 |

 sklearn 

| feature    |   importance |
|------------+--------------|
| int_corr   |    0.082184  |
| amb1_1_f   |    0.0326569 |
| age_f      |    0.0292627 |
| exphappy_f |    0.0277691 |
| fun1_1_f   |    0.027369  |
| income_f   |    0.025696  |
| date_f     |    0.0242891 |
| fun2_1_f   |    0.02369   |
| amb1_1     |    0.023001  |
| tuition_f  |    0.0228923 |


## Задание 5

In [17]:
clf = RandomForestClassifier(n_jobs=-1, random_state=42)
param_dist = {"min_samples_split": range(2, 11),
              "n_estimators": range(1, 101),
              "criterion": ["gini", "entropy"]}
n_iter_search = 50
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)
random_search.fit(X, y)
print(random_search.best_params_)

{'n_estimators': 67, 'min_samples_split': 10, 'criterion': 'entropy'}
